In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [ ]:
# ConversationBufferMemory => 지난대화 모드 저장
# ConversationBufferWindowMemory => 대화 계수만큼만 최근 대화만 저장
# ConversationTokenBufferMemory => 토큰 길이 만큼만 최근 대화 저장함.

# save_context(inputs, outputs) 메서드를 사용하여 대화 기록을 저장할 수 있습니다.
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

# LLM 모델을 생성합니다.
llm = ChatOpenAI(temperature=0)

# ConversationChain을 생성합니다.
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),# ConversationBufferMemory를 사용합니다.
    # memory=ConversationBufferWindowMemory(k=2, return_messages=True), # ConversationBufferWindowMemory 사용 => 최근 2개응답만 저장.
    # memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=150, return_messages=True),  # 최대 토큰 길이를 150개로 제한
)

# 대화를 시작합니다.
response = conversation.predict(
    input="안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
)
print(response)
print(f'--'*20)

# 이전 대화내용을 불렛포인트로 정리해 달라는 요청을 보냅니다.
response1 = conversation.predict(
    input="이전 답변을 불렛포인트 형식으로 정리하여 알려주세요."
)
print(response1)

In [ ]:
# KG 메모리 (지식그래프) 사용
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

llm = ChatOpenAI(temperature=0)

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

# 대화 시작
response = conversation_with_kg.predict(
    input="내이름은 철수입니다. 영희는 내 직장 동료이며, 저와는 초등학교 친구고, 우리회사에 새로운 디자이너입니다."
)


print(response)
print(f'--'*20)

# 영희 에 대한 질문 시 지식그래프를 이용해 영희에 대한 정보를 출력함.
response1=conversation_with_kg.memory.load_memory_variables({"input": "영희는 누구인가요?"})
print(response1)

In [2]:
# ConversationSummaryBufferMemory
# => 대화 요약 버퍼 메모리(요약은 영어로 요약됨)

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

llm = ChatOpenAI(temperature=0)

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template)

summarymemory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,  # 요약의 기준이 되는 토큰 길이를 설정합니다.
    return_messages=True,
)

conversation_summary = ConversationChain(
    llm=llm, prompt=prompt, memory=summarymemory
)

# 대화 시작
query = "내이름은 철수입니다. 영희는 내 직장 동료이며, 저와는 초등학교 친구고, 우리회사에 새로운 디자이너입니다."
response = conversation_summary.predict(
    input=query
)

# 저장
'''
summarymemory.save_context(
    inputs={"human": query},
    outputs={"ai": response},
)
'''
print(f'Q:{query}')
print(f'A: {response}')
print(f'--'*20)

query = "길동도 내 직장 동료이며, 저보다는 선배이며, 우리회사에서 파이썬 프로그램 개발을 담당하고 있습니다."
response = conversation_summary.predict(
    input=query
)
'''
# 저장
summarymemory.save_context(
    inputs={"human": query},
    outputs={"ai": response},
)
'''
print(f'Q:{query}')
print(f'A: {response}')
print(f'--'*20)

query = "오늘은 주간업무회의가 있는 날이어서, 저와 영희, 길동 모두 아침 9시 회의에 참석해야 합니다."
response = conversation_summary.predict(
    input=query
)
'''
# 저장
summarymemory.save_context(
    inputs={"human": query},
    outputs={"ai": response},
)
'''
print(f'Q:{query}')
print(f'A: {response}')
print(f'--'*20)

# 메모리에 저장된 대화내용 확인
print(summarymemory.load_memory_variables({})["history"])


Q:내이름은 철수입니다. 영희는 내 직장 동료이며, 저와는 초등학교 친구고, 우리회사에 새로운 디자이너입니다.
A: 안녕하세요, 철수님! 영희님은 여러모로 당신에게 가까운 분이군요. 초등학교 친구이자 직장 동료이며, 이제는 회사에서도 함께 일하게 되었군요. 새로운 디자이너로서 영희님과 함께 일하게 되어 행복할 것 같습니다. 어떤 프로젝트를 함께 진행하게 되었나요?
----------------------------------------
Q:길동도 내 직장 동료이며, 저보다는 선배이며, 우리회사에서 파이썬 프로그램 개발을 담당하고 있습니다.
A: 저희 회사에서도 파이썬 프로그램 개발을 담당하고 있는 동료가 있어서 반가워요! 여러분이 함께 일하게 된다니 정말 기쁘네요. 프로젝트에 대해 좀 더 알려주실 수 있을까요?
----------------------------------------
Q:오늘은 주간업무회의가 있는 날이어서, 저와 영희, 길동 모두 아침 9시 회의에 참석해야 합니다.
A: 안녕하세요! 저는 여기서 새로운 디자이너인 영희와 함께 일하게 될 철수입니다. 아침 9시에 주간업무회의가 있어서 기대가 되네요. 회의에서는 어떤 프로젝트를 진행할지 알려주시겠어요? 함께 일하게 되어 기쁩니다!
----------------------------------------
[SystemMessage(content='Cheolsoo introduces himself and talks about his colleague Yeonghee, who is a childhood friend and a new designer at their company. The AI responds warmly, acknowledging their close relationship and expressing excitement about working together. The AI also mentions having a colleague at their company who is respons

In [8]:
# VectorStoreRetrieverMemory
# => VectorStoreRetrieverMemory 는 벡터 스토어에 메모리를 저장하고 호출될 때마다 가장 '눈에 띄는' 상위 K개의 문서를 쿼리합니다.

import faiss
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

# LLM 모델 정의
llm = ChatOpenAI(temperature=0)

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template)


# 임베딩 모델을 정의합니다.
embeddings_model = OpenAIEmbeddings()

# Vector Store 를 초기화 합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

from langchain.memory import VectorStoreRetrieverMemory

# 벡터 조회가 여전히 의미적으로 관련성 있는 정보를 반환한다는 것을 보여주기 위해서입니다.
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
memory = VectorStoreRetrieverMemory(retriever=retriever)

conversation = ConversationChain(
    llm=llm, prompt=prompt, memory=memory
)

# 대화 시작
query = "안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다."
response = conversation.predict(
    input=query
)
print(f'Q:{query}')
print(f'A: {response}')
print(f'--'*20)

query = "프로젝트에서 어떤 역할을 맡았나요?"
response = conversation.predict(
    input=query
)
print(f'Q:{query}')
print(f'A: {response}')
print(f'--'*20)

query = "팀 프로젝트에서 어려움을 겪었던 경험이 있다면 어떻게 해결했나요?"
response = conversation.predict(
    input=query
)
print(f'Q:{query}')
print(f'A: {response}')
print(f'--'*20)

query = "개발자로서 자신의 강점은 무엇이라고 생각하나요?"
response = conversation.predict(
    input=query
)
print(f'Q:{query}')
print(f'A: {response}')
print(f'--'*20)

# 메모리에 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.
print(memory.load_memory_variables({"prompt": "프로젝트에서 맡은 역활은 뭔가요?"})["history"])


Q:안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
A: 안녕하세요! 저는 AI입니다. 제 이름은 GPT-3이고, OpenAI에서 개발되었습니다. 저는 자연어 처리 및 기계 학습을 통해 대화를 이해하고 응답합니다. 저는 175억 개의 매개 변수를 가지고 있으며, 다양한 주제에 대해 대화할 수 있습니다. 면접에 참석해주셔서 감사합니다. 저는 항상 새로운 경험을 통해 배우고 성장하고 있습니다. 부족한 부분이 있을 수 있지만 최선을 다해 대화를 이어가겠습니다. 자기소개가 어떻게 도와드릴까요?
----------------------------------------
Q:프로젝트에서 어떤 역할을 맡았나요?
A: 프로젝트에서 제가 맡은 역할은 주로 자연어 처리와 기계 학습 모델의 구축이었습니다. 데이터 수집, 전처리, 모델 학습 및 평가를 담당했고, 결과를 분석하여 프로젝트 팀과 협력하여 최종 결과물을 완성했습니다. 또한, 모델의 성능 향상을 위해 하이퍼파라미터 튜닝과 모델 최적화 작업도 수행했습니다. 이를 통해 프로젝트의 성공에 기여할 수 있었습니다.
----------------------------------------
Q:팀 프로젝트에서 어려움을 겪었던 경험이 있다면 어떻게 해결했나요?
A: 팀 프로젝트에서 어려움을 겪었던 경험이 있었습니다. 그 때는 데이터의 품질이 낮아서 모델의 성능이 좋지 않았습니다. 이를 해결하기 위해 데이터 전처리 과정을 다시 검토하고 추가적인 데이터 수집을 진행했습니다. 또한, 모델의 하이퍼파라미터를 조정하여 성능을 향상시키는 작업을 진행했습니다. 이러한 노력 끝에 문제를 해결하고 프로젝트를 성공적으로 마무리할 수 있었습니다.
----------------------------------------
Q:개발자로서 자신의 강점은 무엇이라고 생각하나요?
A: 제가 개발자로서 가장 큰 강점은 자연어 처리와 기계 학습 분야에 대한 전문 지식과 경험입니다. 이를 바탕으로 프로젝트에서 데이터 수집, 전처리, 모델 학습 및 